In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

loader = TextLoader("./founder.txt")
docs = loader.load()

In [2]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
chunks = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())

In [3]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [4]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [5]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [6]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser

rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
rag_chain.invoke("who is the owner of the restaurant?")

"Chef Amico is the owner of the restaurant. He is known for his dedication, generosity, and passion for food that extends beyond the restaurant's walls. The restaurant quickly gained fame for its authentic flavors and innovative twists on traditional recipes."

### Agents

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [9]:
search = TavilySearchResults()

In [10]:
search.invoke("what is the weather in Munich?")

[{'url': 'https://weather.com/weather/tenday/l/e0c0cd4ab9873fee5066145204f7161a0c1f7468722e633345583634cff0bbd6',
  'content': "Don't Miss\nRadar\nStay Healthy This Winter\nNavigate Flip-Flopping Temps\nThat's Not What Was Expected\nOutside\nEclipse Emergency\nInterstellar Innovations\nFight The Flaking\nWinter Skin Check-In\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n recents\nSpecialty Forecasts\n10 Day Weather-Munich, Bavaria, Germany\nPotential disruption due to extreme low temperatures\nTonight\nSun 25 | Night\nCloudy skies early, then partly cloudy after midnight. Wed 06\nWed 06 | Day\nMostly cloudy skies early will become partly cloudy later in the day. Sun 10\nSun 10 | Day\nPartly cloudy skies. Fri 01\nFri 01 | Day\nShowers in the morning, then cloudy in the afternoon."},
 {'url': 'https://www.accuweather.com/en/de/munich/80331/september-weather/178086',
  'content': 'Get the monthly weather forecast for Mun

Use agent with tools

In [11]:
from langchain.tools.retriever import create_retriever_tool

In [12]:
retriever_tool = create_retriever_tool(
    retriever,
    "restaurant_retriever",
    "Get information about the italian restaurant bella vista",
)

In [13]:
tools = [search, retriever_tool]

In [14]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [17]:
agent_executor.invoke({"input": "Who is the dad in the simpsons?"})

{'input': 'Who is the dad in the simpsons?',
 'output': 'The dad in The Simpsons is Homer J. Simpson. Homer is overweight, lazy, works at a nuclear power plant, and loves doughnuts.'}

In [18]:
agent_executor.invoke({"input": "Who is the owner of the restaurant?"})

{'input': 'Who is the owner of the restaurant?',
 'output': 'The owner of the restaurant Bella Vista is Chef Amico. He is known for creating the restaurant with authentic flavors and innovative twists on traditional recipes.'}

In [19]:
agent_executor.invoke({"input": "How is the weather in Munich? I would like to eat outside"})

{'input': 'How is the weather in Munich? I would like to eat outside',
 'output': 'The current weather in Munich is sunny with a temperature of 28°C (82.4°F). It seems like a great day to eat outside! You can check more details about the weather forecast in Munich [here](https://www.weatherapi.com/).'}

Add memory to agents

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

In [25]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "Who is the owner of the restaurant?"},
    config={"configurable": {"session_id": "session123"}},
)

{'input': 'Who is the owner of the restaurant?',
 'chat_history': [],
 'output': 'The restaurant Bella Vista Italian restaurant was created by Chef Amico. He believed in hospitality as an art form and considered a meal to be a celebration. Patrons at the restaurant were welcomed as part of an extended family.'}

In [27]:
agent_with_chat_history.invoke(
    {"input": "How is the weather in Munich? I would like to eat outside"},
    config={"configurable": {"session_id": "session123"}},
)

{'input': 'How is the weather in Munich? I would like to eat outside',
 'chat_history': [HumanMessage(content='Who is the owner of the restaurant?'),
  AIMessage(content='The restaurant Bella Vista Italian restaurant was created by Chef Amico. He believed in hospitality as an art form and considered a meal to be a celebration. Patrons at the restaurant were welcomed as part of an extended family.')],
 'output': 'The current weather in Munich is sunny with a temperature of 28.0°C (82.4°F). It is a great day to eat outside and enjoy the weather!'}